In [1]:
!pip install pandas
import pandas as pd
import pyarrow.csv
import requests
import pyarrow as pa
import pyarrow.fs
import io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 17.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 KB 42.3 MB/s eta 0:00:00


In [2]:
#Q1: how many live DataNodes are in the cluster?

!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 25260875776 (23.53 GB)
DFS Remaining: 25260826624 (23.53 GB)
DFS Used: 49152 (48 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.20.0.2:9866 (project-4-team-koslon-4-dn-1.project-4-team-koslon-4_default)
Hostname: c5ac4c6f32f3
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 13173837824 (12.27 GB)
DFS Remaining: 12630413312 (11.76 GB)
DFS Used%: 0.00%
DFS Remaining%:

In [3]:
wi = pyarrow.csv.read_csv("hdma-wi-2021.csv")

In [4]:
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv

In [5]:
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://boss:9000/double.csv

In [6]:
#Q2: what are the logical and physical sizes?
!hdfs dfs -du -h hdfs://boss:9000/double.csv hdfs://boss:9000/single.csv

166.8 M  333.7 M  hdfs://boss:9000/double.csv
166.8 M  166.8 M  hdfs://boss:9000/single.csv


In [7]:
#Q2: 

In [8]:
#Q3: what is the file status for single.csv?
url = "http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS"
response = requests.get(url)
if response.status_code == 200:
    file_status = response.json()
    print(file_status)
else:
    print(f"Error {response.status_code}: {response.text}")

{'FileStatus': {'accessTime': 1698426532650, 'blockSize': 1048576, 'childrenNum': 0, 'fileId': 16386, 'group': 'supergroup', 'length': 174944099, 'modificationTime': 1698426538212, 'owner': 'root', 'pathSuffix': '', 'permission': '644', 'replication': 1, 'storagePolicy': 0, 'type': 'FILE'}}


In [9]:
#Q4: what is the location for the first block of single.csv?
url = "http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true"
response = requests.get(url)
if response.status_code == 200:
    file_status = response.json()
    print(file_status["Location"])
else:
    print(f"Error {response.status_code}: {response.text}")

http://c5ac4c6f32f3:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0


In [10]:
#Q5: how are the blocks of single.csv distributed across the two DataNode containers?
url = "http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS"
response = requests.get(url)
if response.status_code == 200:
    file_status = response.json()
    #print(file_status)
else:
    print(f"Error {response.status_code}: {response.text}")

block_locs = {}
key = 'hosts'

block_location_dict = file_status['BlockLocations']

block_location_list = block_location_dict['BlockLocation']

for block in block_location_list:
    host = block[key][0]  # Assuming 'hosts' is a list with a single element
    if host in block_locs:
        block_locs[host] += 1
    else:
        block_locs[host] = 1

block_locs

{'c5ac4c6f32f3': 87, 'e0a32136407f': 80}

In [11]:
len(block_location_list)

167

In [12]:
#Q6: what are the first 10 bytes of single.csv?
hdfs = pa.fs.HadoopFileSystem("boss",9000)
with hdfs.open_input_file("/single.csv") as f:
    first10 = f.read_at(10,0)

first10

2023-10-27 17:09:23,006 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


b'activity_y'


In [13]:
#Q7: how many lines of single.csv contain the string "Single Family"?
with hdfs.open_input_file("/single.csv") as f:
    reader = io.TextIOWrapper(io.BufferedReader(f))
    count = 0
    for i, line in enumerate(reader):
        if "Single Family" in line:
            count += 1
count

444874